In [ ]:
NP_API_KEY = "" # Edit your key here
GOOGLE_MAP_API_KEY = "" # Edit your key here

In [3]:
# !curl 'https://developer.nps.gov/api/v1/parks?stateCode=ca&api_key=NP-Api-Key'

import urllib.request, json
# Configure API request
endpoint = "https://developer.nps.gov/api/v1/parks?stateCode=ca"
HEADERS = {"X-Api-Key":NP_API_KEY}
req = urllib.request.Request(endpoint,headers=HEADERS)

# Execute request and parse response
response = urllib.request.urlopen(req).read()
data = json.loads(response.decode('utf-8'))
data

{'total': '34',
 'limit': '50',
 'start': '0',
 'data': [{'id': 'C08AD828-98FF-478E-A63C-614E7534274B',
   'url': 'https://www.nps.gov/alca/index.htm',
   'fullName': 'Alcatraz Island',
   'parkCode': 'alca',
   'description': "Alcatraz reveals stories of American incarceration, justice, and our common humanity. This small island was once a fort, a military prison, and a maximum security federal penitentiary. In 1969, the Indians of All Tribes occupied Alcatraz for 19 months in the name of freedom and Native American civil rights. We invite you to explore Alcatraz's complex history and natural beauty.",
   'latitude': '37.82676234',
   'longitude': '-122.4230206',
   'latLong': 'lat:37.82676234, long:-122.4230206',
   'activities': [{'id': '1DFACD97-1B9C-4F5A-80F2-05593604799E',
     'name': 'Food'},
    {'id': '0B685688-3405-4E2A-ABBA-E3069492EC50',
     'name': 'Wildlife Watching'},
    {'id': '5A2C91D1-50EC-4B24-8BED-A2E11A1892DF', 'name': 'Birdwatching'},
    {'id': '24380E3F-AD9D-

In [4]:
data['data'][0].keys()

dict_keys(['id', 'url', 'fullName', 'parkCode', 'description', 'latitude', 'longitude', 'latLong', 'activities', 'topics', 'states', 'contacts', 'entranceFees', 'entrancePasses', 'fees', 'directionsInfo', 'directionsUrl', 'operatingHours', 'addresses', 'images', 'weatherInfo', 'name', 'designation', 'relevanceScore'])

In [5]:
# some entranceFees and entrancePasses are empty
data['data']
# remove `directionsUrl`, `name`, `designation`, `relevanceScore`, `latLong` in each park
for x in data['data']:
    x.pop('directionsUrl', None)
    x.pop('name', None)
    x.pop('designation', None)
    x.pop('relevanceScore', None)
    x.pop('latLong', None)


# store all the data['data'] in a json file
with open('ca_np.json', 'w') as outfile:
    json.dump(data['data'], outfile)

In [18]:
data['data'][0]

{'id': 'C08AD828-98FF-478E-A63C-614E7534274B',
 'url': 'https://www.nps.gov/alca/index.htm',
 'fullName': 'Alcatraz Island',
 'parkCode': 'alca',
 'description': "Alcatraz reveals stories of American incarceration, justice, and our common humanity. This small island was once a fort, a military prison, and a maximum security federal penitentiary. In 1969, the Indians of All Tribes occupied Alcatraz for 19 months in the name of freedom and Native American civil rights. We invite you to explore Alcatraz's complex history and natural beauty.",
 'latitude': '37.82676234',
 'longitude': '-122.4230206',
 'latLong': 'lat:37.82676234, long:-122.4230206',
 'activities': [{'id': '1DFACD97-1B9C-4F5A-80F2-05593604799E', 'name': 'Food'},
  {'id': '0B685688-3405-4E2A-ABBA-E3069492EC50', 'name': 'Wildlife Watching'},
  {'id': '5A2C91D1-50EC-4B24-8BED-A2E11A1892DF', 'name': 'Birdwatching'},
  {'id': '24380E3F-AD9D-4E38-BF13-C8EEB21893E7', 'name': 'Shopping'},
  {'id': '467DC8B8-0B7D-436D-A026-80A22358F

In [26]:
# read the json file
import json
with open('ca_np.json') as f:
    data = json.load(f)

In [16]:
data[0].keys()

dict_keys(['id', 'url', 'fullName', 'parkCode', 'description', 'latitude', 'longitude', 'activities', 'topics', 'states', 'contacts', 'entranceFees', 'entrancePasses', 'fees', 'directionsInfo', 'operatingHours', 'addresses', 'images', 'weatherInfo'])

In [29]:
# just keep the `name` in each activity and topics
for x in data:
    x['activities'] = [y['name'] for y in x['activities']]
    x['topics'] = [y['name'] for y in x['topics']]

# clean up the contacts
for x in data:
    x['contacts'] = {"phoneNumber": x['contacts']['phoneNumbers'][0]['phoneNumber'] if len(x['contacts']['phoneNumbers']) > 0 else '',
                     "emailAddress": x['contacts']['emailAddresses'][0]['emailAddress'] if len(x['contacts']['emailAddresses']) > 0 else ''}

## add ratings and reviews from google api

In [34]:
data[3]

{'id': 'B39C368F-CB27-49EC-B2A9-E6C1552430FB',
 'url': 'https://www.nps.gov/cali/index.htm',
 'fullName': 'California National Historic Trail',
 'parkCode': 'cali',
 'description': 'Follow in the footsteps of over 250,000 emigrants who traveled to the gold fields and rich farmlands of California during the 1840s and 1850s: the greatest mass migration in American history. The California National Historic Trail is over 5,000 miles long and covers portions of 10 states. Step into history along more than 1,000 miles of ruts and traces from travelers and their overland wagons.',
 'latitude': '42.3999643979',
 'longitude': '-108.702415369',
 'activities': ['Auto and ATV',
  'Scenic Driving',
  'Guided Tours',
  'Self-Guided Tours - Auto',
  'Hiking',
  'Horse Trekking',
  'Horseback Riding',
  'Museum Exhibits'],
 'topics': ['African American Heritage',
  'Archeology',
  'Arts',
  'Photography',
  'Poetry and Literature',
  'Burial, Cemetery and Gravesite',
  'Colonization and Settlement',
 

In [23]:
import requests

# Replace this list with your list of national parks
national_parks = [x['fullName'] for x in data]

# Your Google Maps API key
api_key = GOOGLE_MAP_API_KEY

def get_place_id(park_name):
    response = requests.get(
        'https://maps.googleapis.com/maps/api/place/findplacefromtext/json',
        params={
            'input': park_name,
            'inputtype': 'textquery',
            'fields': 'place_id',
            'key': api_key
        }
    )
    results = response.json().get('candidates', [])
    if results:
        return results[0].get('place_id')
    return None

def get_park_rating(place_id):
    response = requests.get(
        'https://maps.googleapis.com/maps/api/place/details/json',
        params={
            'place_id': place_id,
            'fields': 'name,rating,reviews',
            'key': api_key
        }
    )
    return response.json().get('result', {})

comments = []
for x in data:
    print(x['fullName'])
    place_id = get_place_id(x['fullName'])
    if place_id:
        details = get_park_rating(place_id)
        print(f"Park: {x['fullName']}")
        print(f"Rating: {details.get('rating')}")
        # x['rating'] = str(details.get('rating'))
        comments = details.get('reviews', [])
        comments.append({'parkId':x["id"], 'comments': comments})
        x['comments'] = len(comments)

    else:
        print(f"Place ID not found for {x['fullName']}")


Alcatraz Island
Park: Alcatraz Island
Rating: 4.7
Butterfield Overland National Historic Trail
Park: Butterfield Overland National Historic Trail
Rating: None
Cabrillo National Monument
Park: Cabrillo National Monument
Rating: 4.8
California National Historic Trail
Park: California National Historic Trail
Rating: 4.4
Castle Mountains National Monument
Park: Castle Mountains National Monument
Rating: 4.6
Channel Islands National Park
Park: Channel Islands National Park
Rating: 4.7
César E. Chávez National Monument
Park: César E. Chávez National Monument
Rating: 4.6
Death Valley National Park
Park: Death Valley National Park
Rating: 4.7
Devils Postpile National Monument
Park: Devils Postpile National Monument
Rating: 4.7
Eugene O'Neill National Historic Site
Park: Eugene O'Neill National Historic Site
Rating: 4.7
Fort Point National Historic Site
Park: Fort Point National Historic Site
Rating: 4.7
Golden Gate National Recreation Area
Park: Golden Gate National Recreation Area
Rating: 4.7

In [24]:
# store all the data in a json file
with open('ca_np.json', 'w') as outfile:
    json.dump(data, outfile)

In [21]:
comments


[{'parkId': 'C08AD828-98FF-478E-A63C-614E7534274B',
  'comments': [{'author_name': 'Ajdin',
    'author_url': 'https://www.google.com/maps/contrib/111722073813499378403/reviews',
    'language': 'en',
    'original_language': 'en',
    'profile_photo_url': 'https://lh3.googleusercontent.com/a-/ALV-UjXXULiVp2094vCjTF7Co9-w0SvdiFYHBmY-9BouvtPEQ_Vb=s128-c0x00000000-cc-rp-mo-ba4',
    'rating': 5,
    'relative_time_description': 'in the last week',
    'text': 'Such an amazing peace of history.\n\nI was really enjoying the visit, I would highly recommend going with the guide once you get there, our guide was awesome, and so knowledgeable.\n\nYou have a museum store where you can buy some goodies to have as a memory of your visit.',
    'time': 1701291212,
    'translated': False},
   {'author_name': 'P C',
    'author_url': 'https://www.google.com/maps/contrib/118358188874722196085/reviews',
    'language': 'en',
    'original_language': 'en',
    'profile_photo_url': 'https://lh3.googleu

In [22]:
with open('comments.json', 'w') as outfile:
    json.dump(comments, outfile)

In [25]:
import time
int(time.time())

1702002310